In [1]:
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np
import onetimepad
print("Imports Successful")

Imports Successful


In [2]:
n= 100

def vPlot(qc):
    tbackend = BasicAer.get_backend('statevector_simulator')
    job = execute(qc, tbackend).result()
    return plot_bloch_multivector(job.get_statevector(qc), title="")

def getResults(qc,shot):
    backend = Aer.get_backend('qasm_simulator')
    shots = shot
    job = execute(qc, backend, shots=shots, memory=True)
    result = job.result()
    memory = result.get_memory()
    oArray = []
    for x in range(0, shots):
        converted = int(memory[x], 2)
        oArray.append(converted)
    return oArray

def encode_message(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # if base[i] = 0
            if bits[i] == 0: # & bit is 0, do nothing
                pass 
            else:
                qc.x(0) #if base = 0 & bit = 1, change it to 0
                
        else: # if base[i] = 1
            if bits[i] == 0: # if base =1, and bit = 0, put bit into super position
                qc.h(0)
            else:
                qc.x(0) #if base = 1 and bit = 1, set bit to 0 and put it in super position
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

def measure_message(message, bases):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # if base[q] = 0, then measure it, return 1 or 0
            message[q].measure(0,0)
        if bases[q] == 1: # if base[q] = 1, then use a H gate, which is a gate for super position
            message[q].h(0)
            message[q].measure(0,0) #measure the message
        result = execute(message[q], backend, shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit) # return values
    return measurements

def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            good_bits.append(bits[q])
    return good_bits

#check bit strings, and remove nonsecure bits used for checking
def sample_bits(bits, selection):
    sample = [] 
    for i in selection:
        i = np.mod(i, len(bits))
        sample.append(bits.pop(i)) #remove sampled selection from key
    return sample

def quantumNumGen (shots):
    qc = QuantumCircuit(1,1)
    qc.h(0)
    qc.measure(0,0)
    result = getResults(qc, shots)
    return result

def keyGen(n,sample_size):
    alice_bits = quantumNumGen(n)
    alice_bases = quantumNumGen(n)
    message = encode_message(alice_bits, alice_bases)
    bob_bases = quantumNumGen(n)
    bob_results = measure_message(message, bob_bases)
    bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
    alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
    sample_size = 15
    bit_selection = randint(n, size=sample_size)
    bob_sample = sample_bits(bob_key, bit_selection)
    alice_sample = sample_bits(alice_key, bit_selection)
    if bob_sample != alice_sample:
        print("Outside interfence detected")
        return
    return bob_key;

In [3]:
key = keyGen(n,25)
print(key)
print(len(key))
print(key)

[0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0]
36
[0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0]


In [4]:
temp = [str(i) for i in key]
#print(temp)
newKey = ("".join(temp))
print(type(newKey))
print(len(newKey))
print(newKey)

<class 'str'>
36
000110101110000100101010010011011000


In [5]:
myString = "the social security number is enc[123-45-678]enc"
start = myString.find("enc[")
end = myString.find("]enc")
print(start, end)

subStart = start + 4
subEnd = end
#print(myString[subStart], myString[subEnd])
mySub = myString[subStart:subEnd]
#print(len(mySub))
#print(mySub)
#print(myString)

30 44


In [6]:
#!pip install onetimepad

In [7]:
print(newKey)
cipher = onetimepad.encrypt(mySub,newKey)
print (cipher)
cipher = onetimepad.decrypt(cipher,newKey)
print (cipher)

000110101110000100101010010011011000
0102031c05051c060609
123-45-678


In [8]:
def sendMessage(s,newKey):
    start = s.find("enc[")
    end = s.find("]enc")
    subStart = start + 4
    subEnd = end
    mySub = s[subStart:subEnd]
    cipher = onetimepad.encrypt(mySub,newKey)
    final = s[:start] + "dec["+cipher+"]dec" + s[(end+4):]
    return final

def receiveMessage(s,newKey):
    start = s.find("dec[")
    end = s.find("]dec")
    subStart = start + 4
    subEnd = end
    mySub = s[subStart:subEnd]
    cipher = onetimepad.decrypt(mySub, newKey)
    final = s[:start] +cipher+ s[(end+4):]
    return final
    

In [9]:
stringIn = "My bank account number is enc[123456789abcd]enc, please delete after reading"
key = keyGen(n,25)
temp = [str(i) for i in key]
myKey = ("".join(temp))
send = sendMessage(stringIn,myKey)
print(send)
receive = receiveMessage(send,myKey)
print(receive)

My bank account number is dec[01030204050607090851535354]dec, please delete after reading
My bank account number is 123456789abcd, please delete after reading
